- docs: https://face-recognition.readthedocs.io/en/latest/face_recognition.html
- face recognition used: https://github.com/ageitgey/face_recognition

## For testing only

In [2]:
import cv2
import face_recognition
""" print(cv2.__version__) """
# yall gottta add ur own images and change the path to the image if u wanna test too.
img = cv2.imread('faces/ryan_reynolds.jpg')
rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_encodings = face_recognition.face_encodings(rgb_img)[0]

img2 = cv2.imread('faces/ryan_reynolds1.jpg')
rgb_img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
img_encodings2 = face_recognition.face_encodings(rgb_img2)[0]

img3 = cv2.imread('faces/ryan_gosling.webp')
rgb_img3 = cv2.cvtColor(img3, cv2.COLOR_BGR2RGB)
img_encodings3 = face_recognition.face_encodings(rgb_img3)[0]

img4 = cv2.imread('faces/ryan_gosling1.webp')
rgb_img4 = cv2.cvtColor(img4, cv2.COLOR_BGR2RGB)
img_encodings4 = face_recognition.face_encodings(rgb_img4)[0]

cv2.imshow('Ryan Reynolds', img)
cv2.imshow('Ryan Reynolds 1', img2)
cv2.imshow('Ryan Gosling', img3)
cv2.waitKey(0)



-1

In [3]:
result = face_recognition.compare_faces([img_encodings], img_encodings2) # Should return True
print(result)

[np.True_]


In [4]:
result = face_recognition.compare_faces([img_encodings3], img_encodings4) # Should return True
print(result)

[np.True_]


In [5]:
result = face_recognition.compare_faces([img_encodings], img_encodings3) # Should return False
print(result)

[np.False_]


In [6]:
result = face_recognition.compare_faces([img_encodings2], img_encodings4) # Should return False
print(result)

[np.False_]


## Using real time video feed using webcam
- create a folder called faces
- add your image to the faces folder and pictures of other people to test

In [7]:
import cv2
import face_recognition
import numpy as np
import math
import os, sys

# Define face_confidence function
def face_confidence(face_distance, face_match_threshold=0.6):
    if face_distance > face_match_threshold:
        range = (1.0 - face_match_threshold)
        linear_val = (1.0 - face_distance) / (range * 2.0)
        return round(linear_val,2) #rounding to 2 decimal places
    else:
        range = face_match_threshold
        linear_val = 1.0 - (face_distance / (range * 2.0))
        return round(linear_val,2)
    
class FaceRecognition:
    face_locations = []
    face_encodings = []
    face_names = []
    known_face_encodings = []
    known_face_names = []
    process_current_frame = True #Reccommended from docs saves processing power not making it detect faces for every frame

    def __init__(self):
        self.encode_faces()

    def encode_faces(self):
        for image in os.listdir('faces'):
            face_image = face_recognition.load_image_file(f'faces/{image}')
            face_enc = face_recognition.face_encodings(face_image)[0]

            self.known_face_encodings.append(face_enc)
            self.known_face_names.append(image)
        print(self.known_face_names)

    def video_recognition(self):
        cap = cv2.VideoCapture(0)
        if not cap.isOpened():
            print('Error: Could not open video.')
            sys.exit()

        while True:
            ret, frame = cap.read()

            if self.process_current_frame:
                small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
                rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

                # Find face in frame
                self.face_locations = face_recognition.face_locations(rgb_small_frame)
                self.face_encodings = face_recognition.face_encodings(rgb_small_frame, self.face_locations)

                #recognition part
                self.face_names = []
                for face_encoding in self.face_encodings:
                    """ match = face_recognition.compare_faces(self.known_face_encodings, face_encoding, tolerance=0.5) """ ## tolerance is the threshold for face matching. This is just for testing
                    match = face_recognition.compare_faces(self.known_face_encodings, face_encoding)
                    # default name and confidence
                    name = '???'
                    confidence = '???'

                    face_distances = face_recognition.face_distance(self.known_face_encodings, face_encoding)
                    best_match_index = np.argmin(face_distances)

                    if match[best_match_index]:
                        name = self.known_face_names[best_match_index]
                        confidence = face_confidence(face_distances[best_match_index])
                    
                    self.face_names.append(f'{name} ({confidence})')

            self.process_current_frame = not self.process_current_frame #only process every other frame

            # Drawing the binding box
            for(top, right, bottom, left), name in zip(self.face_locations, self.face_names):
                top*=4
                right*=4
                bottom*=4
                left*=4

                cv2.rectangle(frame, (left, top), (right, bottom), (255, 0, 0), 2)
                """ cv2.rectangle(frame, (left, bottom-35), (right, bottom), (255, 0, 0), -1) # """
                cv2.putText(frame, name, (left+6, bottom-6), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 1)
            
            cv2.imshow('Face Recognition', frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()



if __name__ == '__main__':
    fr = FaceRecognition()
    fr.video_recognition()

['me.JPG', 'ryan_gosling.webp', 'ryan_gosling1.webp', 'ryan_reynolds.jpg', 'ryan_reynolds1.jpg']
